# Genero indicadores censales para la app

In [155]:
from __future__ import unicode_literals
import pyredatam
import pandas as pd
import os
import hashlib
from simpledbf import Dbf5

In [156]:
filtro_jovenes = "PERSONA.P03 >= 15 AND PERSONA.P03 <= 18"

In [157]:
def get_data(area_list_query):
    hash_query = hashlib.md5(area_list_query).hexdigest()
    file_name = hash_query + ".csv"
    file_path = "data/{}".format(file_name)

    if unicode(file_name) in os.listdir("data"):
        df = pd.read_csv(file_path, encoding="utf8", dtype={"Código": str})
    
    else:
        df = pyredatam.cpv2010arg.make_arealist_query(area_list_query)
        df["Código"] = df["Código"].astype(str)
        df.to_csv(file_path, encoding="utf8", index=False)

    return df.set_index("Código")

In [158]:
provincias = pd.read_excel("data/Unidades geoestadisticas Censo 2010.xls", sheetname="provincias", 
                           converters={"codigo": lambda x: str(x).zfill(2)},
                           names=["codigo", "provincia_nombre"]).set_index("codigo")

In [159]:
unidades = pd.read_excel("data/Unidades geoestadisticas Censo 2010.xls", sheetname="unidades", 
                           converters={"cod_provincia": lambda x: str(x).zfill(2)})

In [160]:
departamentos = Dbf5("data/departamentos/pxdptodatos.dbf").to_dataframe().rename(
    columns={"nombre": "departamento_nombre", "link": "departamento_id"}
).set_index(
    "departamento_id"
)[["departamento_nombre"]]

## Bloque 1

### Por departamento/comuna

¿Qué porcentaje de chicos de 15 a 18 años pensás que vive en tu comuna? ¿Y en la Ciudad de Buenos Aires?

In [161]:
# cantidad de población total, por género
query_pobl_depto = pyredatam.arealist_query("DPTO", "PERSONA.P02")
df_pobl_depto = get_data(query_pobl_depto)

In [162]:
# cantidad de población entre 15 y 18 años, por género
query_pobl_joven_depto = pyredatam.arealist_query(
    "DPTO", "PERSONA.P02", universe_filter=filtro_jovenes)
df_pobl_joven_depto = get_data(query_pobl_joven_depto)

In [163]:
df_depto = pd.DataFrame(index=df_pobl_depto.index)
df_depto["jovenes"] = df_pobl_joven_depto["Total"] / df_pobl_depto["Total"]

¿Qué porcentaje de chicos de 15 a 18 años pensás que asisten a la escuela en tu comuna? ¿Y en la Ciudad de Buenos Aires?

In [164]:
# cantidad de población entre 15 y 18 años, por asistencia escolar
query_asiste_joven_depto = pyredatam.arealist_query("DPTO", "PERSONA.P08", universe_filter=filtro_jovenes)
df_asiste_joven_depto = get_data(query_asiste_joven_depto)

In [165]:
df_depto["jovenes_asistencia"] = df_asiste_joven_depto["Asiste"] / df_pobl_joven_depto["Total"]

### Por provincia/ciudad

¿Qué porcentaje de chicos de 15 a 18 años pensás que vive en tu comuna? ¿Y en la Ciudad de Buenos Aires?

In [166]:
# cantidad de población total, por género
query_pobl_prov = pyredatam.arealist_query("PROV", "PERSONA.P02")
df_pobl_prov = get_data(query_pobl_prov)

In [167]:
# cantidad de población entre 15 y 18 años, por género
query_pobl_joven_prov = pyredatam.arealist_query(
    "PROV", "PERSONA.P02", universe_filter=filtro_jovenes)
df_pobl_joven_prov = get_data(query_pobl_joven_prov)

In [168]:
df_prov = pd.DataFrame(index=df_pobl_prov.index)
df_prov["jovenes"] = df_pobl_joven_prov["Total"] / df_pobl_prov["Total"]

¿Qué porcentaje de chicos de 15 a 18 años pensás que asisten a la escuela en tu comuna? ¿Y en la Ciudad de Buenos Aires?

In [169]:
# cantidad de población entre 15 y 18 años, por asistencia escolar
query_asiste_joven_prov = pyredatam.arealist_query("PROV", "PERSONA.P08", universe_filter=filtro_jovenes)
df_asiste_joven_prov = get_data(query_asiste_joven_prov)

In [170]:
df_prov["jovenes_asistencia"] = df_asiste_joven_prov["Asiste"] / df_pobl_joven_prov["Total"]

## Bloque 2

### Por departamento/comuna

¿Cuántas personas duermen por cuarto en tu casa? ¿Cuántas personas creés que duermen por cuarto, en promedio, en tu comuna? ¿Y en la Ciudad de Buenos Aires?

In [171]:
query = """
RUNDEF Job

DEFINE HOGAR.TOTPERS2
AS COUNT PERSONA
TYPE REAL
RANGE 0-200
VARLABEL "Cantidad de personas por hogar"
DECIMALS 2

DEFINE HOGAR.INDHAC2
AS (HOGAR.TOTPERS2 / HOGAR.H15)
TYPE REAL
RANGE 0-200
VARLABEL "Indice de hacinamiento"
DECIMALS 2

TABLE TABLE1
    AS AREALIST
    OF DPTO, HOGAR.INDHAC2
"""

In [172]:
# habitaciones
query_habs_depto = pyredatam.arealist_query("DPTO", "HOGAR.H15")
df_habs_depto = get_data(query_habs_depto)

In [173]:
# personas x hogar
query_pers_hogar_depto = pyredatam.arealist_query("DPTO", "HOGAR.TOTPERS")
df_pers_hogar_depto = get_data(query_pers_hogar_depto)

In [174]:
df_habs_depto_mul = df_habs_depto[df_habs_depto.columns[:-1]]

In [175]:
for i in range(1, 31):
    df_habs_depto_mul[unicode(i)] = df_habs_depto_mul[unicode(i)] * i

/Users/abenassi/anaconda/envs/odi/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [176]:
cant_cuartos = df_habs_depto_mul.sum(axis=1)

In [177]:
cant_personas = df_pobl_depto.Total

In [178]:
indhac2 = cant_personas / cant_cuartos

In [179]:
# hacinamiento
query_hac_depto = pyredatam.arealist_query("DPTO", "HOGAR.INDHAC")
df_hac_depto = get_data(query_hac_depto)

In [180]:
df_hac_depto2 = df_hac_depto[df_hac_depto.columns[:-1]]

In [181]:
for i, j in zip(range(0, 6), [0.25, 0.75, 1.25, 1.75, 2.5, 4.0]):
    df_hac_depto2[df_hac_depto2.columns[i]] = df_hac_depto2[df_hac_depto2.columns[i]] * j

/Users/abenassi/anaconda/envs/odi/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [182]:
indhac3 = df_hac_depto2.sum(axis=1) / df_hac_depto.Total

In [183]:
df_indhacs = pd.DataFrame({"indhac2": indhac2, "indhac3": indhac3})

In [184]:
df_indhacs.head()

,indhac2,indhac3
Código,,
02001,1.429665,1.192540
02002,1.081067,0.805423
02003,1.362607,1.119662
02004,1.474937,1.271931
02005,1.273984,0.988682


In [185]:
df_depto["indhac2"] = indhac2

¿Vivís en una vivienda alquilada? ¿Qué porcentaje de personas creés que viven en una vivienda alquilada en tu comuna? ¿Y en la Ciudad de Buenos Aires?

In [186]:
# cantidad de población entre 15 y 18 años, que vive en una vivienda alquilada
query_prop_joven_depto = pyredatam.arealist_query("DPTO", "HOGAR.PROP", universe_filter=filtro_jovenes)
df_prop_joven_depto = get_data(query_prop_joven_depto)

In [187]:
df_depto["propiedad"] = df_prop_joven_depto["Inquilino"] / df_prop_joven_depto["Total"]

### Por provincia/ciudad

¿Cuántas personas duermen por cuarto en tu casa? ¿Cuántas personas creés que duermen por cuarto, en promedio, en tu comuna? ¿Y en la Ciudad de Buenos Aires?

In [188]:
# habitaciones
query_habs_prov = pyredatam.arealist_query("PROV", "HOGAR.H15")
df_habs_prov = get_data(query_habs_prov)

In [189]:
# personas x hogar
query_pers_hogar_prov = pyredatam.arealist_query("PROV", "HOGAR.TOTPERS")
df_pers_hogar_prov = get_data(query_pers_hogar_prov)

In [190]:
df_habs_prov_mul = df_habs_prov[df_habs_prov.columns[:-1]]

In [191]:
for i in range(1, 31):
    df_habs_prov_mul[unicode(i)] = df_habs_prov_mul[unicode(i)] * i

/Users/abenassi/anaconda/envs/odi/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [192]:
cant_cuartos = df_habs_prov_mul.sum(axis=1)

In [193]:
cant_personas = df_pobl_prov.Total

In [194]:
indhac2 = cant_personas / cant_cuartos

In [195]:
# hacinamiento
query_hac_prov = pyredatam.arealist_query("PROV", "HOGAR.INDHAC")
df_hac_prov = get_data(query_hac_prov)

In [196]:
df_hac_prov2 = df_hac_prov[df_hac_prov.columns[:-1]]
for i, j in zip(range(0, 6), [0.25, 0.75, 1.25, 1.75, 2.5, 4.0]):
    df_hac_prov2[df_hac_prov2.columns[i]] = df_hac_prov2[df_hac_prov2.columns[i]] * j

/Users/abenassi/anaconda/envs/odi/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [197]:
indhac3 = df_hac_prov2.sum(axis=1) / df_hac_prov.Total

In [198]:
df_indhacs = pd.DataFrame({"indhac2": indhac2, "indhac3": indhac3})

In [199]:
df_indhacs.head()

,indhac2,indhac3
Código,,
02,1.274730,1.007657
06,1.528409,1.283741
10,1.551428,1.376618
14,1.448385,1.166431
18,1.639087,1.449928


In [200]:
df_prov["indhac2"] = indhac2

¿Vivís en una vivienda alquilada? ¿Qué porcentaje de personas creés que viven en una vivienda alquilada en tu comuna? ¿Y en la Ciudad de Buenos Aires?

In [201]:
# cantidad de población entre 15 y 18 años, que vive en una vivienda alquilada
query_prop_joven_prov = pyredatam.arealist_query("PROV", "HOGAR.PROP", universe_filter=filtro_jovenes)
df_prop_joven_prov = get_data(query_prop_joven_prov)

In [202]:
df_prov["propiedad"] = df_prop_joven_prov["Inquilino"] / df_prop_joven_prov["Total"]

## Dumps de la data

In [203]:
df_prov = df_prov.merge(provincias, how="left", right_index=True, left_index=True)
df_prov.index.set_names("provincia_id", inplace=True)

In [204]:
df_depto = df_depto.merge(departamentos, how="left", right_index=True, left_index=True)
df_depto.drop(["94021", "94028"], inplace=True)
df_depto.index.set_names("departamento_id", inplace=True)
df_depto["provincia_id"] = df_depto.index.str[:2]

In [205]:
rename_cols = {"jovenes": "jovenes_proporcion", 
               "jovenes_asistencia": "jovenes_asistencia_escolar",
              "indhac2": "personas_por_cuarto",
              "propiedad": "propietarios_vivienda_proporcion"}
df_prov.rename(columns=rename_cols, inplace=True)
df_depto.rename(columns=rename_cols, inplace=True)

In [206]:
df_prov.to_excel("indicadores/indicadores_provincia.xlsx", index_label="provincia_id")
df_depto.to_excel("indicadores/indicadores_departamento.xlsx", index_label="departamento_id")

In [207]:
df_prov.to_csv("indicadores/indicadores_provincia.csv", index_label="provincia_id", encoding="utf8")
df_depto.to_csv("indicadores/indicadores_departamento.csv", index_label="departamento_id", encoding="utf8")